# OpenCVを使ったトランプカード認識

## matplotlib
matplotlibはグラフ描画が可能なライブラリです。(opencvとは別のライブラリです)

### Import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

### 棒グラフを表示

In [ ]:
x = np.array(range(100))

plt.plot(x,x)

In [ ]:
y = x**2

plt.plot(x,y)

### 散布図

In [ ]:
x = np.random.rand(50)
y = np.random.rand(50)
plt.scatter(x, y)

### 画像を表示

In [ ]:
img = np.array([[[0,0,0],
            [0,255,0],
            [0,0,0]],
           [[0,0,0],
            [0,255,0],
            [0,0,0]],
           [[0,0,0],
            [0,255,0],
            [0,0,0]]])
img = img.transpose(2,1,0)
plt.imshow(img)

## OpenCV
opencvは画像処理・解析用ライブラリです。

次のリンクが基本操作としてわかりやすいです。
+ [OpenCV 入門：画像処理・画像認識・機械学習の実装を徹底解説（全実装コード公開）](https://www.codexa.net/opencv_python_introduction/)

### Installation
標準でインストールされてないはずなのでpipコマンドでインストールします。

`pip install opencv-python`

### トランプ認識

In [ ]:
import cv2

In [ ]:
img = cv2.imread("cards.jpg")
pltimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #BGR -> RGB
plt.xticks([]) #軸の非表示 
plt.yticks([]) #軸の非表示 
plt.imshow(pltimg)

### グレースケール処理
+ [【OpenCV】色空間HSVとグレースケール](https://progzakki.sanachan.com/program-lang/python/python-opencv-color-space-and-grayscale/)

In [ ]:
gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
def cv2_imshow(img):
    cv2.imshow("img",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

cv2_imshow(gimg) #グレースケール処理をした画像を表示

### 閾値処理
目的の図形以外を排除します
+ [画像のしきい値処理](http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html)

In [ ]:
_, threshold = cv2.threshold(gimg, 200, 255, cv2.THRESH_BINARY)
cv2_imshow(threshold)

### 輪郭を抽出

In [ ]:
contours,_ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contour_img = img.copy()
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area < 10000:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True) #輪郭の近似度合を指定
        cv2.drawContours(contour_img, [approx],0, (255,0,0), 2)

cv2_imshow(contour_img)

## 文字認識(OCR)

### Tesseract
OCRエンジン(Google開発)  
**Installation**  
`conda install -c conda-forge tesseract`

### pyocr
↑のエンジンをPythonで使えるようにするライブラリ  
**Installation**  
`conda install -c conda-forge pyocr`

### Import

In [ ]:
import pyocr
import pyocr.builders
from PIL import Image
import os
TESSDATA_PATH = 'C:\\Users\\__\\jupyter\\circ\\tessdata'
os.environ["TESSDATA_PREFIX"] = TESSDATA_PATH
    

In [ ]:
tools = pyocr.get_available_tools()
tool = tools[0]
langs = tool.get_available_languages()
langs

### OpenCVのデータをpyocrに対応させる
+ 参考: https://stackoverflow.com/questions/30404756/how-to-pass-opencv-image-to-tesseract-in-python

In [ ]:
pil_threshold = Image.fromarray(img)
print(f"{type(threshold)}から")
print(type(pil_threshold))

In [ ]:
res = tool.image_to_string(
    pil_threshold,lang="eng",
    builder=pyocr.builders.DigitBuilder(tesseract_layout=6)
    )
print(res[0])